### Weekly Contest 392



#### 3105. Longest Strictly Increasing or Strictly Decreasing Subarray

> You are given an array of integers nums. Return the length of the longest subarray of nums which is either strictly increasing or strictly decreasing.

*Tags: Array*

Solution: Once encountering a new number which is smaller/larger than the previous one, it means it's no longer a strictly increasing or or strictly decreasing array, so we should start considering a new candidate array, i.e. move the start point of subarray.

Optimization: only one time traverse.

In [1]:
from typing import List

In [2]:
class Solution:
    def longestMonotonicSubarray(self, nums: List[int]) -> int:
        l, ans = 0, 1
        for i in range(1, len(nums)):
            if nums[i] > nums[i-1]:
                ans = max(ans, i-l+1)
            else:
                l = i
        l = 0
        for i in range(1, len(nums)):
            if nums[i] < nums[i-1]:
                ans = max(ans, i-l+1)
            else:
                l = i
        return ans

In [3]:
class Solution:
    def longestMonotonicSubarray(self, nums: List[int]) -> int:
        incr, decr, ans = 1, 1, 1
        prev = nums[0]
        for num in nums:
            incr = incr * (num > prev) + 1
            decr = decr * (num < prev) + 1
            prev = num
            ans = max(ans, incr, decr)     
        return ans

#### 3106. Lexicographically Smallest String After Operations With Constraint

> You are given a string s and an integer k.
>
> Define a function distance(s1, s2) between two strings s1 and s2 of the same length n as:
>
> The sum of the minimum distance between s1[i] and s2[i] when the characters from 'a' to 'z' are placed in a cyclic order, for all i in the range [0, n - 1].
> For example, distance("ab", "cd") == 4, and distance("a", "z") == 1.
>
> You can change any letter of s to any other lowercase English letter, any number of times.
>  
> Return a string denoting the lexicographically smallest string t you can get after some changes, such that distance(s, t) <= k.

*Tags: String, Greedy*


Solution: try to make the beginning characters as close to 'a' as possible

In [4]:
class Solution: 
    def computeDistance(self, c, k):
        cost = min(ord(c) - ord('a'), ord('z') - ord(c) + 1) 
        if cost <= k : return 'a', k-cost
        else: return chr(ord(c) - k), 0
    
    def getSmallestString(self, s: str, k: int) -> str:
        s = list(s)
        for i,c in enumerate(s):
            s[i], k = self.computeDistance(c, k)
            if k == 0: break
        return "".join(s)

#### 3106. Lexicographically Smallest String After Operations With Constraint

> You are given a string s and an integer k.
>
> Define a function distance(s1, s2) between two strings s1 and s2 of the same length n as:
>
> The sum of the minimum distance between s1[i] and s2[i] when the characters from 'a' to 'z' are placed in a cyclic order, for all i in the range [0, n - 1].
> For example, distance("ab", "cd") == 4, and distance("a", "z") == 1.
>
> You can change any letter of s to any other lowercase English letter, any number of times.
>  
> Return a string denoting the lexicographically smallest string t you can get after some changes, such that distance(s, t) <= k.

*Tags: Array, Greedy, Sorting*

Solution: This question is easy as it told you there will always be only one median. So just sort the numbers(2*10^5), make sure all the numbers to the left are smaller than k, and to the right are larger than k.

In [5]:
class Solution:
    def minOperationsToMakeMedianK(self, nums: List[int], k: int) -> int:
        nums.sort()            
        median = len(nums)//2
        ans = 0
        
        for i in range(len(nums)):
            if i < median and nums[i] > k: ans +=  nums[i]  - k 
            elif i > median and nums[i]  < k: ans += k - nums[i] 
            elif i == median: ans += abs(k - nums[i])     
    
        return ans

#### 3107. Minimum Operations to Make Median of Array Equal to K

> There is an undirected weighted graph with n vertices labeled from 0 to n - 1.
>
> You are given the integer n and an array edges, where edges[i] = [ui, vi, wi] indicates that there is an edge between vertices ui and vi with a weight of wi.
> 
> A walk on a graph is a sequence of vertices and edges. The walk starts and ends with a vertex, and each edge connects the vertex that comes before it and the vertex that comes after it. It's important to note that a walk may visit the same edge or vertex more than once.
> 
> The cost of a walk starting at node u and ending at node v is defined as the bitwise AND of the weights of the edges traversed during the walk. In other words, if the sequence of edge weights encountered during the walk is w0, w1, w2, ..., wk, then the cost is calculated as w0 & w1 & w2 & ... & wk, where & denotes the bitwise AND operator.
> 
> You are also given a 2D array query, where query[i] = [si, ti]. For each query, you need to find the minimum cost of the walk starting at vertex si and ending at vertex ti. If there exists no such walk, the answer is -1.
> 
> Return the array answer, where answer[i] denotes the minimum cost of a walk for query i.

*Tags: Array, Bit Manipulation, Union Find, Graph*

Observation:
1. The more paths you traverse, the better result you may get, as a_1 & a_2 & a_3... & a_n <= min(a_1 & a_2 & a_3... & a_n)
2. If we call those connected nodes as a group, then the shortest cost between any two nodes in this group is the same. Given observation#1, you should always try using as many edges as you can. So the best strategy is to try using every edge you are able to achieve.

Solution: As we have observed that, all the elements in a group are sharing some common features, then union find is very suitable for such situation.

Attention: Be caseful to adopt path compresson, otherwise you would be faced with TLE.

In [6]:
class Solution:  
    
    def __init__(self):
        self.uf = None
        self.value = None
    
    def find(self, x):
        tmp = []
        while self.uf[x] != x:
            tmp.append(x)
            x = self.uf[x]
        #path compresson
        for t in tmp:
            self.uf[t] = x
        return x
    
    def union(self, x, y):
        xx = self.find(x)
        yy = self.find(y)
        self.uf[xx] = self.uf[x] = self.uf[y] = yy
        self.values[yy] &= self.values[xx]
        return yy
    
    def minimumCost(self, n: int, edges: List[List[int]], query: List[List[int]]) -> List[int]:
        self.uf = [i for i in range(n)]
        self.values = [int(pow(2,32))-1] * n
        
        for u, v, w in edges:
            self.values[self.union(u, v)] &= w
            
        ans = []
        for u, v in query:
            if self.find(u) != self.find(v):
                ans.append(-1)
            else:
                ans.append(self.values[self.find(u)])
        
        return ans